# Decoding an aristocrat faster

The way we have decoded aristocrats so far is effective, but slow. 

In [1]:
from utils import *

First we read in the text we are going to encode (Emancipation proclamation), and make reference counts of word pair frequencies (from the bible).

In [2]:
eman_letters = make_letters('../data/Emancipation_Proclamation.txt')
ciphertext = (encode_aris(eman_letters, make_rand_aris_key()))
bible_letters = make_letters('../data/bible.txt')
bible_letter_count = count_letters(bible_letters)
bible_letter_percent = normalize_counts_no_spaces(bible_letter_count)
bible_pair_counts = count_letter_pairs(bible_letters)
bible_matrix = compute_transition_matrix(bible_pair_counts, 0.5)

The faster method is based on pre-computing counts of letter pairs in the encoded ciphertext. 


In [3]:
ciphertext_pair_counts = count_letter_pairs(ciphertext)

Now create our start key, and run the decoder:

In [4]:
eman_pair_counts = count_letter_pairs(eman_letters)

start_key = make_start_key(ciphertext, bible_letter_percent)
start_key.append(26)
print(start_key)
print(''.join(decode_aris(ciphertext, start_key)))
num_key = new_decode_aris_faster(start_key, ciphertext_pair_counts, bible_matrix)

print(start_key)
print(num_key)


[19, 23, 2, 7, 6, 0, 12, 4, 9, 18, 17, 5, 3, 1, 13, 25, 16, 22, 24, 14, 20, 8, 15, 10, 11, 21, 26]
khamhsw  h tshanusiytioabw tre ysenideat of tre maited nthten of hgesiuhh ysoulhghtioapresehn oa tre tpeatwneuoad dhw of neytegbes ia tre wehs of oms losd oae tromnhad eicrt rmadsed had nijtwtpo h ysoulhghtioa phn innmed bw tre ysenideat of tre maited nthten uoathiaiac hgoac otres triacn tre follopiac to pittrht oa tre fisnt dhw of khamhsw ia tre wehs of oms losd oae tromnhad eicrt rmadsed had nijtwtrsee hll yesnoan reld hn nlhven pitria haw nthte os denicahted yhst of h nthte tre yeoyle preseof nrhll trea be ia sebellioa hchiant tre maited nthten nrhll be trea treauefosphsd had foseves fsee had tre ejeumtive covesageat of tre maited nthten iaulmdiac tre gilithsw had ahvhl hmtrositw treseof pill seuocaixe had ghiathia tre fseedog of nmur yesnoan had pill do ao hut os hutn to seysenn nmur yesnoan os haw of treg ia haw effostn trew ghw ghze fos treis hutmhl fseedogtrht tre ejeumtive pill oa

In [5]:
print(''.join(decode_aris(ciphertext, num_key)))

kpqlphz  p ohpqswhaeoayqcz onf ehfsamfqo yd onf lqaofm sopofs yd pvfhawpp ehywtpvpoayqinfhfps yq onf oifqozsfwyqm mpz yd sfeofvcfh aq onf zfph yd ylh tyhm yqf onylspqm fagno nlqmhfm pqm sajozoiy p ehywtpvpoayq ips asslfm cz onf ehfsamfqo yd onf lqaofm sopofs wyqopaqaqg pvyqg yonfh onaqgs onf dyttyiaqg oy iaoonpo yq onf dahso mpz yd kpqlphz aq onf zfph yd ylh tyhm yqf onylspqm fagno nlqmhfm pqm sajozonhff ptt efhsyqs nftm ps stpufs iaonaq pqz sopof yh mfsagqpofm epho yd p sopof onf efyetf infhfyd snptt onfq cf aq hfcfttayq pgpaqso onf lqaofm sopofs snptt cf onfq onfqwfdyhiphm pqm dyhfufh dhff pqm onf fjfwloauf gyufhqvfqo yd onf lqaofm sopofs aqwtlmaqg onf vataophz pqm qpupt plonyhaoz onfhfyd iatt hfwygqabf pqm vpaqopaq onf dhffmyv yd slwn efhsyqs pqm iatt my qy pwo yh pwos oy hfehfss slwn efhsyqs yh pqz yd onfv aq pqz fddyhos onfz vpz vprf dyh onfah pwolpt dhffmyvonpo onf fjfwloauf iatt yq onf dahso mpz yd kpqlphz pdyhfspam cz ehywtpvpoayq mfsagqpof onf sopofs pqm ephos yd sopofs ad pqz

In [6]:
old = compute_key_log_likelihood_pairs(ciphertext_pair_counts, bible_matrix, start_key)
new = compute_key_log_likelihood_pairs(ciphertext_pair_counts, bible_matrix, num_key)
print(new - old)
old = find_pair_log_likelihood(decode_aris(ciphertext, start_key), bible_matrix)
new = find_pair_log_likelihood(decode_aris(ciphertext, num_key), bible_matrix)
print(new - old)
print(start_key)
print(num_key)

-6978.799550054737
-6978.799550054804
[19, 23, 2, 7, 6, 0, 12, 4, 9, 18, 17, 5, 3, 1, 13, 25, 16, 22, 24, 14, 20, 8, 15, 10, 11, 21, 26]
[9, 10, 23, 0, 11, 6, 2, 24, 25, 18, 17, 3, 7, 22, 14, 4, 19, 21, 1, 5, 8, 12, 20, 16, 13, 15, 26]


In [7]:
test_text = eman_letters
changed_text = switch_letters(eman_letters, 'b', 'c')
text_letter_count = count_letters(test_text)
text_pair_count = count_letter_pairs(test_text)
one = find_pair_log_likelihood(test_text, bible_matrix)
two = find_pair_log_likelihood(changed_text, bible_matrix)
diff = (two - one)
start_key = char_to_num(list(alpha_list))
index1 = alpha_list.index("b")
index2 = alpha_list.index("c")
faster_calc = (calculate_log_likelihood_change(test_text, text_pair_count, bible_letter_percent, bible_matrix, index1, index2, start_key))


In [8]:
print(diff)
print(faster_calc)
print(index1)
print(index2)

-220.29815349189448
-220.2981534919021
1
2


In [9]:
random.seed(1)
print(random.randint(0, 25))

4
